# Imports and loading

In [ ]:
from __future__ import annotations
# Standard imports
import os
import torch
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.ActivationCache import ActivationCache
from transformer_lens.HookedTransformer import HookedTransformer
from neel_plotly import line, imshow, scatter
import itertools
from functools import partial
from typing import Callable, Optional, Sequence, Tuple, Union, overload

import einops
import pandas as pd
import torch
from jaxtyping import Float, Int
from tqdm.auto import tqdm
from typing_extensions import Literal

import types
from transformer_lens.utils import Slice, SliceInput

import functools
import re
from collections import defaultdict
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory

# Imports for displaying vis in Colab / notebook

torch.set_grad_enabled(False)

# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

import torch
from collections import defaultdict

# from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

import random

# Loading gemma 2 2b and sae I'm interested in

In [ ]:
os.environ["HF_TOKEN"] = "<hf token here>"
model = HookedSAETransformer.from_pretrained("google/gemma-2-2b", device = device)

sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res", # <- Release name 
    sae_id = "layer_8/width_16k/average_l0_71", # <- SAE id (not always a hook point!)
    device = device
)

# Investigating and exploring the Abstract feature 

In [ ]:
from IPython.display import IFrame
html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

def get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)

## I discovered the feature when investigating arithmetic equations

## If you just look at the top activating logits, its just "=" tokens

In [ ]:
html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=int(15191))
display(IFrame(html, width=1200, height=300))

## But that's not it, when I played with it, I found that it activates on equations that dont have the = token

## Specifically, it needs both operators and operands in the equation to activate

# Generating Clean and corrupted samples

In [ ]:

def generate_example_pair():
    # Generate two random numbers between 1 and 3 digits
    num1 = random.randint(10, 99)
    num2 = random.randint(10, 99)
    
    # Create clean and corrupted examples
    clean_example = f'What is the output of {num1} plus {num2} ? '
    corrupted_example = f'What is the output of {num1} and {num2} ? '
    
    return clean_example, corrupted_example

def generate_dataset(N):
    dataset = []
    for _ in range(N):
        clean, corrupted = generate_example_pair()
        dataset.append((clean, corrupted))
    return dataset

# Example usage
N = 100  # Number of pairs to generate
dataset = generate_dataset(N)

# Print the dataset
for i, (clean, corrupted) in enumerate(dataset):
    print(f"Pair {i+1}:")
    print(f"  Clean:     {clean}")
    print(f"  Corrupted: {corrupted}")
    print()
    if i>10:
        break
        
clean_pr = []
corr_pr = []
for i, (clean, corrupted) in enumerate(dataset):
    clean_pr.append(clean)
    corr_pr.append(corrupted)

## Testing feature activation difference between clean and corrupted prompts

In [ ]:
def run_model_till_feature(prompt):
    _, cache = model.run_with_cache(
            prompt, 
            stop_at_layer=sae.cfg.hook_layer + 1, 
            names_filter=[sae.cfg.hook_name])
    sae_in = cache[sae.cfg.hook_name]
    feature_acts = sae.encode(sae_in).squeeze()
    return feature_acts[:, 15191][-2:].sum()

for i, (clean, corrupted) in enumerate(dataset):
    print("Clean: ", run_model_till_feature(clean))
    print("Corrupted: ", run_model_till_feature(corrupted))

# Run with hooks and cache

For normal patching experiments, the metric is something like the logit difference. In this case, the metric is the activation of a feature. 

As the goal is to find a circuit for a feature, I only care about the components before that feature.

I edited the run_with_cache method to allow addition of an extra hook.

In [ ]:
   
def run_with_cache_with_extra_hook(
    self,
    *model_args: Any,
    current_activation_name: str,
    current_hook: Any,
    names_filter: NamesFilter = None,
    device: DeviceType = None,
    remove_batch_dim: bool = False,
    incl_bwd: bool = False,
    reset_hooks_end: bool = True,
    clear_contexts: bool = False,
    pos_slice: Optional[Union[Slice, SliceInput]] = None,
    **model_kwargs: Any,
):
    """
    Runs the model and returns the model output and a Cache object.
    
    Adds an extra forward hook (current_activation_name, current_hook) to the hooks.

    Args:
        *model_args: Positional arguments for the model.
        current_activation_name: The name of the activation to hook.
        current_hook: The hook function to use.
        names_filter (NamesFilter, optional): A filter for which activations to cache.
        device (str or torch.Device, optional): The device to cache activations on.
        remove_batch_dim (bool, optional): If True, removes the batch dimension when caching.
        incl_bwd (bool, optional): If True, caches gradients as well.
        reset_hooks_end (bool, optional): If True, removes all hooks added by this function.
        clear_contexts (bool, optional): If True, clears hook contexts whenever hooks are reset.
        pos_slice: The slice to apply to the cache output. Defaults to None.
        **model_kwargs: Keyword arguments for the model.

    Returns:
        tuple: A tuple containing the model output and a Cache object.
    """

    pos_slice = Slice.unwrap(pos_slice)

    # Get the caching hooks
    cache_dict, fwd, bwd = self.get_caching_hooks(
        names_filter,
        incl_bwd,
        device,
        remove_batch_dim=remove_batch_dim,
        pos_slice=pos_slice,
    )

    # Add the extra forward hook
    fwd_hooks = [(current_activation_name, current_hook)] + fwd

    # Run the model with the hooks
    with self.hooks(
        fwd_hooks=fwd_hooks,
        bwd_hooks=bwd,
        reset_hooks_end=reset_hooks_end,
        clear_contexts=clear_contexts,
    ):
        model_out = self(*model_args, **model_kwargs)
        if incl_bwd:
            model_out.backward()

    return model_out, cache_dict



# Attach the new method to the model instance
model.run_with_cache_with_extra_hook = types.MethodType(run_with_cache_with_extra_hook, model)




def generic_activation_patch(
    model: HookedTransformer,
    corrupted_tokens: Int[torch.Tensor, "batch pos"],
    clean_cache: ActivationCache,
    patching_metric: Callable[[Float[torch.Tensor, "batch pos d_vocab"]], Float[torch.Tensor, ""]],
    patch_setter: Callable[
        [CorruptedActivation, Sequence[int], ActivationCache], PatchedActivation
    ],
    activation_name: str,
    index_axis_names: Optional[Sequence[AxisNames]] = None,
    index_df: Optional[pd.DataFrame] = None,
    return_index_df: bool = False,
) -> Union[torch.Tensor, Tuple[torch.Tensor, pd.DataFrame]]:
    """
    A generic function to do activation patching, will be specialised to specific use cases.

    Activation patching is about studying the counterfactual effect of a specific activation between a clean run and a corrupted run. The idea is have two inputs, clean and corrupted, which have two different outputs, and differ in some key detail. Eg "The Eiffel Tower is in" vs "The Colosseum is in". Then to take a cached set of activations from the "clean" run, and a set of corrupted.

    Internally, the key function comes from three things: A list of tuples of indices (eg (layer, position, head_index)), a index_to_act_name function which identifies the right activation for each index, a patch_setter function which takes the corrupted activation, the index and the clean cache, and a metric for how well the patched model has recovered.

    The indices can either be given explicitly as a pandas dataframe, or by listing the relevant axis names and having them inferred from the tokens and the model config. It is assumed that the first column is always layer.

    This function then iterates over every tuple of indices, does the relevant patch, and stores it

    Args:
        model: The relevant model
        corrupted_tokens: The input tokens for the corrupted run
        clean_cache: The cached activations from the clean run
        patching_metric: A function from the model's output logits to some metric (eg loss, logit diff, etc)
        patch_setter: A function which acts on (corrupted_activation, index, clean_cache) to edit the activation and patch in the relevant chunk of the clean activation
        activation_name: The name of the activation being patched
        index_axis_names: The names of the axes to (fully) iterate over, implicitly fills in index_df
        index_df: The dataframe of indices, columns are axis names and each row is a tuple of indices. Will be inferred from index_axis_names if not given. When this is input, the output will be a flattened tensor with an element per row of index_df
        return_index_df: A Boolean flag for whether to return the dataframe of indices too

    Returns:
        patched_output: The tensor of the patching metric for each patch. By default it has one dimension for each index dimension, via index_df set explicitly it is flattened with one element per row.
        index_df *optional*: The dataframe of indices
    """

    if index_df is None:
        assert index_axis_names is not None

        # Get the max range for all possible axes
        max_axis_range = {
            "layer": model.cfg.n_layers,
            "pos": corrupted_tokens.shape[-1],
            "head_index": model.cfg.n_heads,
        }
        max_axis_range["src_pos"] = max_axis_range["pos"]
        max_axis_range["dest_pos"] = max_axis_range["pos"]
        max_axis_range["head"] = max_axis_range["head_index"]

        # Get the max range for each axis we iterate over
        index_axis_max_range = [max_axis_range[axis_name] for axis_name in index_axis_names]

        # Get the dataframe where each row is a tuple of indices
        index_df = transformer_lens.patching.make_df_from_ranges(index_axis_max_range, index_axis_names)

        flattened_output = False
    else:
        # A dataframe of indices was provided. Verify that we did not *also* receive index_axis_names
        assert index_axis_names is None
        index_axis_max_range = index_df.max().to_list()

        flattened_output = True

    # Create an empty tensor to show the patched metric for each patch
    if flattened_output:
        patched_metric_output = torch.zeros(len(index_df), device=model.cfg.device)
    else:
        patched_metric_output = torch.zeros(index_axis_max_range, device=model.cfg.device)

    # A generic patching hook - for each index, it applies the patch_setter appropriately to patch the activation
    def patching_hook(corrupted_activation, hook, index, clean_activation):
        return patch_setter(corrupted_activation, index, clean_activation)

    # Iterate over every list of indices, and make the appropriate patch!
    for c, index_row in enumerate(tqdm((list(index_df.iterrows())))):
        index = index_row[1].to_list()

        # The current activation name is just the activation name plus the layer (assumed to be the first element of the input)
        current_activation_name = utils.get_act_name(activation_name, layer=index[0])

        # The hook function cannot receive additional inputs, so we use partial to include the specific index and the corresponding clean activation
        current_hook = partial(
            patching_hook,
            index=index,
            clean_activation=clean_cache[current_activation_name],
        )
        
#         incl_bwd = False
#         cache_dict, fwd, bwd = model.get_caching_hooks(
#             incl_bwd=incl_bwd,
#             device=device,
#             names_filter=None
#         )
        
#         fwd_hooks = [(current_activation_name, current_hook)] + fwd
        # Run the model with the patching hook and get the logits!
        # patched_logits, patched_cache = "", ""
        
        patched_logits, patched_cache = model.run_with_cache_with_extra_hook(
            corrupted_tokens, 
            current_activation_name=current_activation_name, 
            current_hook= current_hook
        )
        # print(patched_cache.keys())
        # print(patched_logits.shape)

        # Calculate the patching metric and store
        if flattened_output:
            patched_metric_output[c] = patching_metric(patched_cache).item()
        else:
            patched_metric_output[tuple(index)] = patching_metric(patched_cache).item()

    if return_index_df:
        return patched_metric_output, index_df
    else:
        return patched_metric_output

def layer_pos_patch_setter(corrupted_activation, index, clean_activation):
    """
    Applies the activation patch where index = [layer, pos]

    Implicitly assumes that the activation axis order is [batch, pos, ...], which is true of everything that is not an attention pattern shaped tensor.
    """
    assert len(index) == 2
    layer, pos = index
    corrupted_activation[:, pos, ...] = clean_activation[:, pos, ...]
    return corrupted_activation
    
get_act_patch_resid_pre = partial(
    generic_activation_patch,
    patch_setter=layer_pos_patch_setter,
    activation_name="resid_pre",
    index_axis_names=("layer", "pos"),
)

def layer_head_vector_patch_setter(
    corrupted_activation,
    index,
    clean_activation,
):
    """
    Applies the activation patch where index = [layer,  head_index]

    Implicitly assumes that the activation axis order is [batch, pos, head_index, ...], which is true of all attention head vector activations (q, k, v, z, result) but *not* of attention patterns.
    """
    assert len(index) == 2
    layer, head_index = index
    corrupted_activation[:, :, head_index] = clean_activation[:, :, head_index]

    return corrupted_activation

get_act_patch_attn_head_out_all_pos = partial(
    generic_activation_patch,
    patch_setter=layer_head_vector_patch_setter,
    activation_name="z",
    index_axis_names=("layer", "head"),
)

def layer_pos_head_vector_patch_setter(
    corrupted_activation,
    index,
    clean_activation,
):
    """
    Applies the activation patch where index = [layer, pos, head_index]

    Implicitly assumes that the activation axis order is [batch, pos, head_index, ...], which is true of all attention head vector activations (q, k, v, z, result) but *not* of attention patterns.
    """
    assert len(index) == 3
    layer, pos, head_index = index
    corrupted_activation[:, pos, head_index] = clean_activation[:, pos, head_index]
    return corrupted_activation

get_act_patch_attn_head_out_by_pos = partial(
    generic_activation_patch,
    patch_setter=layer_pos_head_vector_patch_setter,
    activation_name="z",
    index_axis_names=("layer", "pos", "head"),
)


# METRIC FOR PATCHING 

In [ ]:
def equal_feature_metric(cache):
    sae_in = cache[sae.cfg.hook_name]
    feature_acts = sae.encode(sae_in)
    # print(feature_acts.shape)
    feature_acts = feature_acts.squeeze()
    return feature_acts[:, :, 15191][-2:].sum()


In [ ]:
clean_tokens = model.to_tokens(clean_pr)
corrupted_tokens = model.to_tokens(corr_pr)

_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [ ]:

resid_pre_act_patch_results = get_act_patch_resid_pre(model, corrupted_tokens, clean_cache, equal_feature_metric)

fig = imshow(
    resid_pre_act_patch_results, 
    yaxis="Layer", 
    xaxis="Position", 
    x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
    title="resid_pre Activation Patching",
    return_fig=True  # This ensures the figure object is returned
)

fig.write_image("results/v1/resid_pre_activation_patching0.png")

In [ ]:
attn_head_out_all_pos_act_patch_results = get_act_patch_attn_head_out_all_pos(model, corrupted_tokens, clean_cache, equal_feature_metric)
fig = imshow(attn_head_out_all_pos_act_patch_results,  
       yaxis="Layer", 
       xaxis="Head", 
       title="attn_head_out Activation Patching (All Pos)", 
        return_fig=True)

fig.write_image("results/v1/attn_head_out Activation Patching All Pos0.png")

In [ ]:
DO_SLOW_RUNS = True
ALL_HEAD_LABELS = [f"L{i}H{j}" for i in range(model.cfg.n_layers) for j in range(model.cfg.n_heads)]
if DO_SLOW_RUNS:
    attn_head_out_act_patch_results = get_act_patch_attn_head_out_by_pos(model, corrupted_tokens, clean_cache, equal_feature_metric)
    attn_head_out_act_patch_results = einops.rearrange(attn_head_out_act_patch_results, "layer pos head -> (layer head) pos")
    fig = imshow(attn_head_out_act_patch_results, 
        yaxis="Head Label", 
        xaxis="Pos", 
        x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
        y=ALL_HEAD_LABELS,
        title="attn_head_out Activation Patching By Pos", 
        return_fig=True)
    fig.write_image("results/v1/attn_head_out_act_patch_results0.png")

In [ ]:

# Assuming attn_head_out_act_patch_results is your tensor
sliced_results = attn_head_out_act_patch_results[:72, -7:]
# Adjust the y-axis labels for the first 72 elements
sliced_y_labels = ALL_HEAD_LABELS[:72]

# Adjust the x-axis labels for the last 7 positions
sliced_x_labels = [f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))][-7:]

fig = imshow(
    sliced_results, 
    yaxis="Head Label", 
    xaxis="Pos", 
    x=sliced_x_labels,
    y=sliced_y_labels,
    title="attn_head_out Activation Patching By Pos", 
    width=1000,  # Increase the width of the figure
    height=1200,  # Increase the height of the figure
    return_fig=True
)

# Optionally, you can adjust the tickfont size for better readability
fig.update_layout(
    yaxis=dict(tickfont=dict(size=10)),  # Adjust the size as needed
    xaxis=dict(tickfont=dict(size=10))   # Adjust the size as needed
)

# Save the figure
fig.write_image("results/v1/attn_head_out_act_patch_results_sliced0.png")

import torch

# Assuming sliced_results is your sliced tensor
mean_value = sliced_results.mean().item()
std_dev = sliced_results.std().item()

# Calculate the threshold for one standard deviation away from the mean
lower_threshold = mean_value - std_dev
upper_threshold = mean_value + std_dev

# Identify the indices where the values are one standard deviation away from the mean
indices = (sliced_results < lower_threshold) | (sliced_results > upper_threshold)
y_indices, x_indices = torch.where(indices)

# Extract the corresponding y labels, x labels, and values
tuples_list = [
    (sliced_y_labels[y_idx], sliced_x_labels[x_idx], sliced_results[y_idx, x_idx].item())
    for y_idx, x_idx in zip(y_indices, x_indices)
]

# Display the tuples
print(tuples_list)


# Function to convert L2H1 format to (2, 1)
def convert_to_tuple(layer_head_str):
    layer = int(layer_head_str[1])
    head = int(layer_head_str[3])
    return (layer, head)

# Convert the first element of each tuple in the list
converted_tuples = [convert_to_tuple(item[0]) for item in tuples_list] #[(convert_to_tuple(item[0]), item[1], item[2]) for item in tuples_list]
import json
# Display the result
print("tuples", converted_tuples)
output_path = f"results/v1/converted_tuples0.json"
with open(output_path, 'w') as f:
    json.dump(converted_tuples, f)


def save_relevant_attention_patterns(clean_cache, layer_head_tuples):
    for layer_ind, head_ind in layer_head_tuples:
        temp_att_pattern = clean_cache[f'blocks.{layer_ind}.attn.hook_pattern'][1, head_ind, :, :]
        attention_pattern = temp_att_pattern.detach().cpu().numpy()
        # Define the x and y labels, assuming they correspond to tokens
        tokens = model.to_str_tokens(clean_tokens[0])
        labels = [f"{tok} {i}" for i, tok in enumerate(tokens)]

        # Generate the heatmap
        fig = px.imshow(
            attention_pattern,
            labels=dict(x="Head Position", y="Head Position", color="Attention"),
            x=labels,
            y=labels,
            title=f"Attention Pattern in Layer {layer_ind}, Head {head_ind}",
            color_continuous_scale="Blues"
        )
        # Display the figure
        fig.write_image(f"results/v1/heads/L{layer_ind}H{head_ind}_atten_pattern0.png")
        
save_relevant_attention_patterns(clean_cache, converted_tuples)

https://www.neuronpedia.org/list/cltetkb9g000111bn9cun5dk6


https://www.neuronpedia.org/list/clthkc9p20001ueje4kc0jeoa



# THE ORIGINAL (failed) GOAL: Finding feature circuits for features 

In [ ]:
cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 6.1795, 0.0000, 0.0000, 7.0834],
       device='cuda:0')

In [ ]:
for i, (clean, corrupted) in enumerate(dataset):
    _, clean_cache = model.run_with_cache_with_saes(clean, saes=saes)
    print(f"Pair {i+1}:")
    print(f"  Clean:     {clean}")
    print(clean_cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191][13])
    _, corr_cache = model.run_with_cache_with_saes(corrupted, saes=saes)
    print(f"  Corrupted: {corrupted}")
    print(corr_cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191][13])

Pair 1:
  Clean:     What is the output of 96 plus 43 ? 
tensor(7.0159, device='cuda:0')
  Corrupted: What is the output of 96 and 43 ? 
tensor(0., device='cuda:0')
Pair 2:
  Clean:     What is the output of 56 plus 77 ? 
tensor(5.5384, device='cuda:0')
  Corrupted: What is the output of 56 and 77 ? 
tensor(0., device='cuda:0')
Pair 3:
  Clean:     What is the output of 93 plus 56 ? 
tensor(6.7896, device='cuda:0')
  Corrupted: What is the output of 93 and 56 ? 
tensor(0., device='cuda:0')
Pair 4:
  Clean:     What is the output of 29 plus 37 ? 
tensor(5.6018, device='cuda:0')
  Corrupted: What is the output of 29 and 37 ? 
tensor(0., device='cuda:0')
Pair 5:
  Clean:     What is the output of 29 plus 75 ? 
tensor(6.8103, device='cuda:0')
  Corrupted: What is the output of 29 and 75 ? 
tensor(0., device='cuda:0')
Pair 6:
  Clean:     What is the output of 48 plus 18 ? 
tensor(8.9317, device='cuda:0')
  Corrupted: What is the output of 48 and 18 ? 
tensor(2.8513, device='cuda:0')
Pair 7:
  Clean:     What is the output of 91 plus 12 ? 
tensor(6.7175, device='cuda:0')
  Corrupted: What is the output of 91 and 12 ? 
tensor(0., device='cuda:0')
Pair 8:
  Clean:     What is the output of 33 plus 76 ? 
tensor(6.0717, device='cuda:0')
  Corrupted: What is the output of 33 and 76 ? 
tensor(0., device='cuda:0')
Pair 9:
  Clean:     What is the output of 53 plus 83 ? 
tensor(6.7595, device='cuda:0')
  Corrupted: What is the output of 53 and 83 ? 
tensor(0., device='cuda:0')
Pair 10:
  Clean:     What is the output of 39 plus 16 ? 
tensor(7.5431, device='cuda:0')
  Corrupted: What is the output of 39 and 16 ? 
tensor(0., device='cuda:0')


In [ ]:
# LOAD till 8 
saes = []
for layer in range(9):
    sae, cfg_dict, sparsity = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res", 
        sae_id=closest_strings[layer],  
        device=device
    )
    sae.use_error_term = True
    saes.append(sae)

In [ ]:
def get_deep_attr(obj: Any, path: str):
    """Helper function to get a nested attribute from a object.
    In practice used to access HookedTransformer HookPoints (eg model.blocks[0].attn.hook_z)
    Args:
        obj: Any object. In practice, this is a HookedTransformer (or subclass)
        path: str. The path to the attribute you want to access. (eg "blocks.0.attn.hook_z")

    returns:
        Any. The attribute at the end of the path
    """
    parts = path.split(".")
    # Navigate to the last component in the path
    for part in parts:
        if part.isdigit():  # This is a list index
            obj = obj[int(part)]
        else:  # This is an attribute
            obj = getattr(obj, part)
    return obj

def replace_with_zeros(layer, feature_ind, clean_cache):
    """
    Forward hook function to replace activations in the corrupted run
    with zeros instead of the clean activations.
    
    Args:
        layer (int): The layer number where the intervention should take place.
        feature_ind (int): The feature index to replace.
        clean_cache (ActivationCache): The cache from the clean run (not used in this case).
    """
    def hook_fn(module, input, output):
        # Create a tensor of zeros with the same shape as the specific output slice
        zero_tensor = torch.zeros_like(output[0, :, feature_ind])
        
        # Replace the corresponding activations in the output with zeros
        output[0, :, feature_ind] = zero_tensor
        
        return output
    
    return hook_fn

# Define the hook function
def replace_with_clean_activations(layer, feature_ind, clean_cache):
    """
    Forward hook function to replace activations in the corrupted run
    with those from the clean run.
    
    Args:
        layer (int): The layer number where the intervention should take place.
        feature_ind (int): The feature index to replace.
        clean_cache (ActivationCache): The cache from the clean run.
    """
    def hook_fn(module, input, output):
        # Extract the clean activations
        clean_acts = clean_cache[f'blocks.{layer}.hook_resid_post.hook_sae_acts_post'][0, :, feature_ind]
        # print(output[:, :, feature_ind])
        # print(clean_acts)
        # Replace the corresponding activations in the output of the corrupted run
        output[0, :, feature_ind] = clean_acts
        # print(output[0, :, feature_ind])
        return output
    
    return hook_fn

def subtract_and_mean(tensor1, tensor2):
    """
    Subtract two tensors element-wise and return the mean of the resulting tensor.
    
    Args:
        tensor1 (torch.Tensor): The first tensor.
        tensor2 (torch.Tensor): The second tensor.
        
    Returns:
        torch.Tensor: The mean of the element-wise subtraction of the two tensors.
    """
    # Ensure that both tensors have the same shape
    assert tensor1.shape == tensor2.shape, "Tensors must have the same shape"
    
    # Subtract the tensors element-wise
    difference = tensor1 - tensor2
    
    # Calculate the mean of the difference
    mean_difference = torch.mean(difference)
    
    return mean_difference



def feature_acts(cache):
    return cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191][13]

def overall_feature(cache):
    return cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191]

def feature_difference(cache1, cache2):
    return feature_acts(cache1) - feature_acts(cache2)

def overall_feature_difference(cache1, cache2):
    return subtract_and_mean(cache1['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191], cache2['blocks.8.hook_resid_post.hook_sae_acts_post'][0, :, 15191])
# # Specify the layer number and feature index
# layer = 2  # Example: replace activations at layer 2
# feature_ind = 5  # Example: replace activations at feature index 5




op_clean, clean_cache = model.run_with_cache_with_saes(clean, saes=saes)
op_corr, corr_cache = model.run_with_cache_with_saes(corrupted, saes=saes)

print("og logit", op_clean.shape)
print("Original feature diff: ", feature_difference(clean_cache, corr_cache))

for key, val in operator_features.items():
    layer = key
    for ind in range(10000):
        feature_ind = ind
        hook = replace_with_clean_activations(layer, feature_ind, clean_cache)
        # with model.saes(saes=saes, reset_saes_end=True):
            # Now that SAEs are attached, access the hook point
        hook_point = get_deep_attr(saes[layer], 'hook_sae_acts_post')
        # print(hook_point)
        # Register the hook
        hook_handle = hook_point.register_forward_hook(hook)
        # print(hook_handle)
        # Run the corrupted prompt with the hook applied
        inter_op, corr_cache_intervened = model.run_with_cache_with_saes(corrupted, saes=saes)
        # print("intervened logit", inter_op[0, -1, :10])
        # Remove the hook after the forward pass
        hook_handle.remove()
        
        # print(subtract_and_mean(op_corr, inter_op))
        if overall_feature_difference(corr_cache_intervened, corr_cache) != 0: 
            print("UPPPPPPPP")
        # print(overall_feature_difference(corr_cache_intervened, corr_cache))
            
            
        # hook_point = get_deep_attr(model, f'blocks.{layer}.hook_resid_post.hook_sae_acts_post')
        # print(hook_point)
        # hook_handle = hook_point.register_forward_hook(hook)
        # print(hook_handle)
        # _, corr_cache_intervened = model.run_with_cache_with_saes(corrupted, saes=saes)
        # hook_handle.remove()
        # print("PLESAAE CHANGE ONCE JUST ONCE :", feature_acts(corr_cache_intervened))
        # print("PLESAAE CHANGE ONCE JUST ONCE :", feature_difference(clean_cache, corr_cache_intervened))


In [ ]:
# Corr -> clean 

_, clean_cache = model.run_with_cache_with_saes(clean, saes=saes)
_, corr_cache = model.run_with_cache_with_saes(corrupted, saes=saes)

print("Original feature difference: ", feature_difference(clean_cache, corr_cache))

# Dictionary to store the differences
differences = []

for key, val in operator_features.items():
    layer = key
    print("layer: ", layer)
    for feature_ind in range(1000):
        hook = replace_with_clean_activations(layer, feature_ind, corr_cache)
        
        # Now that SAEs are attached, access the hook point
        hook_point = get_deep_attr(saes[layer], 'hook_sae_acts_post')
        
        # Register the hook
        hook_handle = hook_point.register_forward_hook(hook)
        
        # Run the corrupted prompt with the hook applied
        _, clean_cache_intervened = model.run_with_cache_with_saes(clean, saes=saes)
        
        # Remove the hook after the forward pass
        hook_handle.remove()
        
        # Calculate the overall feature difference
        diff = overall_feature_difference(clean_cache_intervened, clean_cache)
        
        # Store the difference along with the corresponding layer and feature index
        differences.append((layer, feature_ind, diff))

# Sort the differences to get the top 10
top_differences = sorted(differences, key=lambda x: x[2], reverse=True)[:10]

# Print the top 10 features with their differences
print("Top 10 features with the highest overall_feature_difference:")
for layer, feature_ind, diff in top_differences:
    print(f"Layer: {layer}, Feature Index: {feature_ind}, Overall Feature Difference: {diff}")


Original feature difference:  tensor(7.5431, device='cuda:0')
layer:  0
layer:  1
layer:  2
layer:  3
layer:  4
layer:  5
layer:  6
layer:  7
layer:  8
Top 10 features with the highest overall_feature_difference:
Layer: 1, Feature Index: 172, Overall Feature Difference: 5.880991693629767e-07
Layer: 1, Feature Index: 202, Overall Feature Difference: 5.722046125811175e-07
Layer: 5, Feature Index: 697, Overall Feature Difference: 5.722046125811175e-07
Layer: 5, Feature Index: 112, Overall Feature Difference: 5.404154990173993e-07
Layer: 0, Feature Index: 133, Overall Feature Difference: 5.245208853921213e-07
Layer: 2, Feature Index: 616, Overall Feature Difference: 5.245208853921213e-07
Layer: 1, Feature Index: 534, Overall Feature Difference: 5.086263286102621e-07
Layer: 4, Feature Index: 646, Overall Feature Difference: 5.086263286102621e-07
Layer: 2, Feature Index: 498, Overall Feature Difference: 4.92731771828403e-07
Layer: 4, Feature Index: 920, Overall Feature Difference: 4.92731771828403e-07

In [ ]:
# Corr -> clean 

op_clean, clean_cache = model.run_with_cache_with_saes(clean, saes=saes)
op_corr, corr_cache = model.run_with_cache_with_saes(corrupted, saes=saes)

print("og logit", op_clean.shape)
print("Original feature diff: ", feature_difference(clean_cache, corr_cache))

for key, val in operator_features.items():
    layer = key
    for feature_ind in val:
        feature_ind = feature_ind.item()
        hook = replace_with_zeros(layer, feature_ind, corr_cache)
        # with model.saes(saes=saes, reset_saes_end=True):
            # Now that SAEs are attached, access the hook point
        hook_point = get_deep_attr(saes[layer], 'hook_sae_acts_post')
        # print(hook_point)
        # Register the hook
        hook_handle = hook_point.register_forward_hook(hook)
        # print(hook_handle)
        # Run the corrupted prompt with the hook applied
        inter_op, clean_cache_intervened = model.run_with_cache_with_saes(clean, saes=saes)
        # print("intervened logit", inter_op[0, -1, :10])
        # Remove the hook after the forward pass
        hook_handle.remove()
        
        # print(subtract_and_mean(op_corr, inter_op))
        # print("PLESAAE CHANGE ONCE JUST ONCE :", feature_difference(clean_cache_intervened, corr_cache))
        if overall_feature_difference(clean_cache_intervened, clean_cache) != 0: 
            print("UPPPPPPPP")
            print(overall_feature_difference(clean_cache_intervened, clean_cache))
            # print("Og :", overall_feature(clean_cache))
            # print("Intervened :", overall_feature(clean_cache_intervened))

og logit torch.Size([1, 15, 256000])
Original feature diff:  tensor(7.5431, device='cuda:0')
UPPPPPPPP
tensor(5.8810e-07, device='cuda:0')
UPPPPPPPP
tensor(2.3842e-07, device='cuda:0')
UPPPPPPPP
tensor(4.6094e-07, device='cuda:0')
UPPPPPPPP
tensor(6.8347e-07, device='cuda:0')
UPPPPPPPP
tensor(3.0200e-07, device='cuda:0')
UPPPPPPPP
tensor(-1.5895e-08, device='cuda:0')
UPPPPPPPP
tensor(1.5895e-07, device='cuda:0')
UPPPPPPPP
tensor(7.9473e-08, device='cuda:0')
UPPPPPPPP

# Finding the feature 

In [ ]:
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res", # <- Release name 
    sae_id = "layer_8/width_16k/average_l0_71", # <- SAE id (not always a hook point!)
    device = device
)

In [ ]:
from transformer_lens.utils import test_prompt

prompt = "What is the output of 53 plus 34 ? It is "
answer = '8'
# Show that the model can confidently predict the next token.
test_prompt(prompt, answer, model)

In [ ]:
prompts = ["What’s the output of 54 plus 32? It is ", 
"What result is 54 plus 32? It is ",
"What does 54 plus 32 equal? It is ",
"What’s the sum of 54 and 32? It is ",
"What do you get from 54 plus 32? It is ",
"What does 54 plus 32 give? It is ",
"What is 54 plus 32 equal to? It is "]

In [ ]:
import torch
from collections import defaultdict

# Dictionary to accumulate the sums of values for each index
vals_dict = defaultdict(float)
# Dictionary to count occurrences of each index
count_dict = defaultdict(int)

# Run through each prompt and accumulate the values for each index
for pr in prompts:
    _, cache = model.run_with_cache_with_saes(pr, saes=[sae])
    vals, inds = torch.topk(cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, -1, :], 15)
    
    for val, ind in zip(vals, inds):
        vals_dict[ind.item()] += val.item()
        count_dict[ind.item()] += 1

# Calculate the average value for each index
avg_vals_dict = {ind: vals_dict[ind] / count_dict[ind] for ind in vals_dict}

# Sort the indices by their average values in descending order
sorted_avg_vals = sorted(avg_vals_dict.items(), key=lambda x: x[1], reverse=True)

# Get the top 10 indices with the highest average values
top_10_inds = sorted_avg_vals[:10]

# Print the top 10 indices and their corresponding average values
print("Top 10 feature indices and their average values:")
for ind, avg_val in top_10_inds:
    print(f"Index: {ind}, Average Value: {avg_val}")
    html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=ind)
    display(IFrame(html, width=1200, height=300))


In [ ]:
# hooked SAE Transformer will enable us to get the feature activations from the SAE
_, cache = model.run_with_cache_with_saes(prompt, saes=[sae])

print([(k, v.shape) for k,v in cache.items() if "sae" in k])

# note there were 11 tokens in our prompt, the residual stream dimension is 768, and the number of SAE features is 768

from IPython.display import IFrame
html_template = "https://neuronpedia.org/{}/{}/{}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

def get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=0):
    return html_template.format(sae_release, sae_id, feature_idx)

# Create the line plot
fig = px.line(
    cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, -1, :].cpu().numpy(),
    title="Feature activations at the final token position",
    labels={"index": "Feature", "value": "Activation"},
)

# Show the plot (optional, for interactive display)
fig.show()

# Save the figure to a file
fig.write_image("feature_activations.png")  # Save as a PNG image

# let's print the top 5 features and how much they fired
vals, inds = torch.topk(cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, -1, :], 15)
for val, ind in zip(vals, inds):
    print(f"Feature {ind} fired {val:.2f}")
    # html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=ind)
    # display(IFrame(html, width=1200, height=300))

Pos
Feature 16100 fired 27.27
Feature 2024 fired 12.52
Feature 10744 fired 10.61
Feature 8857 fired 9.67
Feature 13307 fired 8.00
Feature 5326 fired 7.58
Feature 8025 fired 7.53
Feature 14739 fired 6.78
Feature 15191 fired 6.25
Feature 15600 fired 6.15
Feature 11484 fired 5.83
Feature 6179 fired 5.61
Feature 2121 fired 5.11
Feature 5927 fired 5.10
Feature 3173 fired 4.64

neg 
Feature 16100 fired 26.37
Feature 2024 fired 12.97
Feature 13307 fired 10.09
Feature 14739 fired 9.11
Feature 8857 fired 8.37
Feature 10744 fired 8.32
Feature 15191 fired 7.78
Feature 5326 fired 7.65
Feature 8025 fired 6.88
Feature 2121 fired 5.81
Feature 15600 fired 5.77
Feature 302 fired 5.42
Feature 4150 fired 5.32
Feature 6179 fired 5.24
Feature 324 fired 5.22

In [ ]:
# let's look at the biggest features in terms of absolute difference
prompt = ["What is 54 plus 32 ? \nIt is ", "What is 54 plus 32 ? \nWhat is "]
_, cache = model.run_with_cache_with_saes(prompt, saes=[sae])
print([(k, v.shape) for k,v in cache.items() if "sae" in k])

feature_activation_df = pd.DataFrame(cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, -1, :].cpu().numpy(),
                                     index = [f"feature_{i}" for i in range(sae.cfg.d_sae)],
)
feature_activation_df.columns = ["int"]
feature_activation_df["string"] = cache['blocks.8.hook_resid_post.hook_sae_acts_post'][1, -1, :].cpu().numpy()
feature_activation_df["diff"]= feature_activation_df["int"] - feature_activation_df["string"]

fig = px.line(
    feature_activation_df,
    title="Feature activations for the prompt",
    labels={"index": "Feature", "value": "Activation"},
)

# hide the x-ticks
fig.update_xaxes(showticklabels=False)
fig.show()
fig.write_image("diff_feature_activations.png") 

diff = cache['blocks.8.hook_resid_post.hook_sae_acts_post'][1, -1, :].cpu() - cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, -1, :].cpu()
vals, inds = torch.topk(torch.abs(diff), 15)
for val, ind in zip(vals, inds):
    print(f"Feature {ind} had a difference of {val:.2f}")
    html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=ind)
    display(IFrame(html, width=1200, height=300))

# Finding other abstract features attempt

In [ ]:
equal_variation = {'What is the output of 53 plus 32 ? A: ': 13,
                   'How much is 12 plus 89 ? A: ': 12, 
                   'What is 31 plus 16? It is ': 12, 
                   'What is the sum of 59 and the product of 82 and 33? It is ': 21,
                   'What is the difference between 19 and the product of 55 and 10 ?' : 21, 
                   '13 + 45 * 42 = ': 12} 
for pr in equal_variation.keys():
    tokens = model.to_str_tokens(pr)
    print(tokens)
    for ind, token in enumerate(tokens):
        # print(ind, token)
        if '?' in token or '=' in token:
            print(ind, token)
            equal_variation[pr] = ind
    # print(model.tokenizer.encode(pr))
            
import torch.nn.functional as F
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res", # <- Release name 
    sae_id = "layer_8/width_16k/average_l0_71", # <- SAE id (not always a hook point!)
    device = device
)
pr = 'What is the output of 53 plus 32 ? A: '

# for ind in range(1, 17):
ind = 13
_, cache = model.run_with_cache_with_saes(pr, saes=[sae])
# print*F.softmax(x, dim=1)
x = cache['blocks.8.hook_resid_post.hook_sae_acts_post']
topk_values, topk_indices = torch.topk(x[0, ind, :], 20)
gathered_values = x[0, :, topk_indices]
softmaxed_values = F.softmax(gathered_values, dim=0)
print(softmaxed_values)
# vals, inds = torch.topk(cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, 13, :], 50)
# inds

# layer = 8
sae, cfg_dict, sparsity = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res",  # <- Release name 
        sae_id=f"layer_8/width_16k/average_l0_71",  # <- SAE id (not always a hook point!)
        device=device
    )
vals_dict = defaultdict(float)
count_dict = defaultdict(int)
for pr, ind in equal_variation.items():
    _, cache = model.run_with_cache_with_saes(pr, saes=[sae])
    vals, inds = torch.topk(cache[f'blocks.8.hook_resid_post.hook_sae_acts_post'][0, ind, :], 50)
    
    for val, ind in zip(vals, inds):
        vals_dict[ind.item()] += val.item()
        count_dict[ind.item()] += 1
print(count_dict)

In [ ]:
# Dictionary to store top 10 features for each layer
top_features_per_layer = {}

for layer in range(model.cfg.n_layers):
    # Load the SAE for the current layer
    sae, cfg_dict, sparsity = SAE.from_pretrained(
        release="gemma-scope-2b-pt-res-canonical",  # <- Release name 
        sae_id=f"layer_{layer}/width_16k/canonical",  # <- SAE id (not always a hook point!)
        device=device
    )
    
    # Dictionary to accumulate the sums of values for each index
    vals_dict = defaultdict(float)
    # Dictionary to count occurrences of each index
    count_dict = defaultdict(int)

    # Run through each prompt and accumulate the values for each index
    for pr, ind in equal_variation.items():
        _, cache = model.run_with_cache_with_saes(pr, saes=[sae])
        vals, inds = torch.topk(cache[f'blocks.{layer}.hook_resid_post.hook_sae_acts_post'][0, ind, :], 15)

        for val, ind in zip(vals, inds):
            vals_dict[ind.item()] += val.item()
            count_dict[ind.item()] += 1

    # Calculate the average value for each index
    avg_vals_dict = {ind: vals_dict[ind] / count_dict[ind] for ind in vals_dict}

    # Sort the indices by their average values in descending order
    sorted_avg_vals = sorted(avg_vals_dict.items(), key=lambda x: x[1], reverse=True)

    # Get the top 10 indices with the highest average values
    top_10_inds = sorted_avg_vals[:5]

    # Store the top 10 features for this layer
    top_features_per_layer[layer] = top_10_inds

    # Optionally, print the top 10 indices and their corresponding average values
    print(f"Top 10 feature indices for layer {layer} and their average values:")
    for ind, avg_val in top_10_inds:
        print(f"Index: {ind}, Average Value: {avg_val}")

Top 10 feature indices for layer 0 and their average values:
Index: 11566, Average Value: 46.200286102294925
Index: 6210, Average Value: 34.4622917175293
Index: 5737, Average Value: 26.257030487060547
Index: 443, Average Value: 25.61747932434082
Index: 8418, Average Value: 25.00115203857422
Top 10 feature indices for layer 1 and their average values:
Index: 14339, Average Value: 52.61330922444662
Index: 6210, Average Value: 40.77787780761719
Index: 9727, Average Value: 32.45850467681885
Index: 1112, Average Value: 23.716838836669922
Index: 1508, Average Value: 21.153118133544922
Top 10 feature indices for layer 2 and their average values:
Index: 9433, Average Value: 36.74750518798828
Index: 15046, Average Value: 29.509745279947918
Index: 1473, Average Value: 25.7916259765625
Index: 324, Average Value: 24.814545822143554
Index: 12653, Average Value: 22.311517906188964
Top 10 feature indices for layer 3 and their average values:
Index: 4793, Average Value: 38.174590301513675
Index: 10909, Average Value: 32.39120864868164
Index: 10538, Average Value: 25.340720494588215
Index: 7465, Average Value: 21.02108383178711
Index: 6210, Average Value: 17.363277435302734
Top 10 feature indices for layer 4 and their average values:
Index: 14260, Average Value: 41.60628128051758
Index: 11465, Average Value: 30.917710876464845
Index: 9433, Average Value: 29.14781951904297
Index: 2983, Average Value: 21.2492618560791
Index: 14307, Average Value: 17.716243743896484
Top 10 feature indices for layer 5 and their average values:
Index: 14260, Average Value: 32.957908630371094
Index: 4793, Average Value: 29.707582092285158
Index: 2841, Average Value: 23.889563242594402
Index: 4463, Average Value: 20.049205780029297
Index: 5971, Average Value: 19.761066436767578
Top 10 feature indices for layer 6 and their average values:
Index: 14525, Average Value: 28.608153978983562
Index: 13401, Average Value: 27.663322067260744
Index: 13506, Average Value: 21.753177642822266
Index: 4315, Average Value: 20.728521982828777
Index: 1227, Average Value: 17.006886800130207
Top 10 feature indices for layer 7 and their average values:
Index: 3186, Average Value: 38.38584518432617
Index: 15356, Average Value: 28.452518463134766
Index: 6751, Average Value: 27.4490966796875
Index: 9234, Average Value: 25.660961151123047
Index: 9182, Average Value: 23.671207427978516
Top 10 feature indices for layer 8 and their average values:
Index: 14801, Average Value: 41.51571083068848
Index: 7357, Average Value: 33.20413398742676
Index: 3178, Average Value: 31.21910858154297
Index: 4781, Average Value: 26.748559188842773
Index: 8217, Average Value: 23.01729393005371
Top 10 feature indices for layer 9 and their average values:
Index: 3517, Average Value: 36.59803263346354
Index: 785, Average Value: 26.311880111694336
Index: 13664, Average Value: 24.884449005126953
Index: 13125, Average Value: 20.077545166015625
Index: 13084, Average Value: 19.83129119873047
Top 10 feature indices for layer 10 and their average values:
Index: 11772, Average Value: 43.767255783081055
Index: 13146, Average Value: 33.37391662597656
Index: 12541, Average Value: 31.429155985514324
Index: 8915, Average Value: 27.96062660217285
Index: 2710, Average Value: 27.930782318115234
Top 10 feature indices for layer 11 and their average values:
Index: 12930, Average Value: 39.17667706807455
Index: 892, Average Value: 39.01293754577637
Index: 7330, Average Value: 31.008285522460938
Index: 14117, Average Value: 30.257923762003582
Index: 3300, Average Value: 23.18971824645996
Top 10 feature indices for layer 12 and their average values:
Index: 10708, Average Value: 47.934686024983726
Index: 9467, Average Value: 29.514047622680664
Index: 6558, Average Value: 28.683330917358397
Index: 15437, Average Value: 28.182409286499023
Index: 4514, Average Value: 21.887222290039062
Top 10 feature indices for layer 13 and their average values:
Index: 3883, Average Value: 50.04939206441244
Index: 5472, Average Value: 45.397453943888344
Index: 9467, Average Value: 42.304283142089844
Index: 12931, Average Value: 38.15758209228515
Index: 12969, Average Value: 29.625812530517578
Top 10 feature indices for layer 14 and their average values:
Index: 15603, Average Value: 58.318609873453774
Index: 2884, Average Value: 42.05490417480469
Index: 15559, Average Value: 39.046241760253906
Index: 3211, Average Value: 32.775944519042966
Index: 15471, Average Value: 29.37887954711914
Top 10 feature indices for layer 15 and their average values:
Index: 4300, Average Value: 51.74925676981608
Index: 15892, Average Value: 42.46276702880859
Index: 772, Average Value: 34.68086624145508
Index: 13987, Average Value: 33.935614013671874
Index: 14211, Average Value: 32.66618982950846
Top 10 feature indices for layer 16 and their average values:
Index: 7645, Average Value: 53.15576171875
Index: 6094, Average Value: 52.22842343648275
Index: 1281, Average Value: 51.58769098917643
Index: 318, Average Value: 48.4905039469401
Index: 16333, Average Value: 42.81694030761719
Top 10 feature indices for layer 17 and their average values:
Index: 9962, Average Value: 81.1238276163737
Index: 6396, Average Value: 65.58076477050781
Index: 8675, Average Value: 46.98172505696615
Index: 14016, Average Value: 43.80245780944824
Index: 2844, Average Value: 42.20041688283285
Top 10 feature indices for layer 18 and their average values:
Index: 12865, Average Value: 88.5863431294759
Index: 11871, Average Value: 81.81711196899414
Index: 2464, Average Value: 62.58462905883789
Index: 623, Average Value: 48.85605239868164
Index: 8486, Average Value: 41.510746765136716
Top 10 feature indices for layer 19 and their average values:
Index: 619, Average Value: 65.64395141601562
Index: 9837, Average Value: 56.29365158081055
Index: 15530, Average Value: 48.74580383300781
Index: 13266, Average Value: 47.22393544514974
Index: 6897, Average Value: 42.807013511657715
Top 10 feature indices for layer 20 and their average values:
Index: 3013, Average Value: 88.45135752360027
Index: 13363, Average Value: 64.82990264892578
Index: 13260, Average Value: 56.37356376647949
Index: 9268, Average Value: 50.00233395894369
Index: 10032, Average Value: 47.21528625488281
Top 10 feature indices for layer 21 and their average values:
Index: 13608, Average Value: 96.21867370605469
Index: 10140, Average Value: 79.54266357421875
Index: 3461, Average Value: 65.96608924865723
Index: 12089, Average Value: 56.30696487426758
Index: 5099, Average Value: 53.99234619140625
Top 10 feature indices for layer 22 and their average values:
Index: 8273, Average Value: 78.92124303181966
Index: 5079, Average Value: 78.50505065917969
Index: 8483, Average Value: 71.00369644165039
Index: 261, Average Value: 69.07780456542969
Index: 1072, Average Value: 68.29635492960612
Top 10 feature indices for layer 23 and their average values:
Index: 1541, Average Value: 181.09188588460287
Index: 5148, Average Value: 88.39195251464844
Index: 6895, Average Value: 81.28666687011719
Index: 9039, Average Value: 72.08507792154948
Index: 12660, Average Value: 65.19308471679688
Top 10 feature indices for layer 24 and their average values:
Index: 13752, Average Value: 127.97515869140625
Index: 8648, Average Value: 87.73385620117188
Index: 11687, Average Value: 80.15138626098633
Index: 3074, Average Value: 75.44028854370117
Index: 15615, Average Value: 75.26011505126954
Top 10 feature indices for layer 25 and their average values:
Index: 13749, Average Value: 202.93736012776694
Index: 1689, Average Value: 182.919921875
Index: 3017, Average Value: 147.8154312133789
Index: 5553, Average Value: 135.17332458496094
Index: 10550, Average Value: 127.17289733886719


In [ ]:
# Iterate through the dictionary and print the indices and corresponding layer
for layer, top_features in top_features_per_layer.items():
    print(f"Layer {layer}:")
    if layer!=8:
        continue
    for ind, avg_val in top_features:
        print(f"  Index: {ind}, Average Value: {avg_val}")
        html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id=f"{layer}-gemmascope-res-16k", feature_idx=ind)
        display(IFrame(html, width=1200, height=300))
    # if layer>20:
    #     break

In [ ]:

sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res", # <- Release name 
    sae_id = "layer_8/width_16k/average_l0_71", # <- SAE id (not always a hook point!)
    device = device
)
pr = 'What is the output of 53 plus 32 ? A: '

# for ind in range(1, 17):
ind = 13
_, cache = model.run_with_cache_with_saes(pr, saes=[sae])
x = cache['blocks.8.hook_resid_post.hook_sae_acts_post']
topk_values, topk_indices = torch.topk(x[0, ind, :], 50)
gathered_values = x[0, :, topk_indices]
print(gathered_values.shape)
# softmaxed_values = F.softmax(gathered_values, dim=0)
# print(softmaxed_values)
# vals, inds = torch.topk(cache['blocks.8.hook_resid_post.hook_sae_acts_post'][0, 13, :], 50)
# inds
softmaxed_x = F.softmax(gathered_values, dim=0)
top3_indices_after_softmax = torch.topk(softmaxed_x, 2, dim=0).indices
max_indices = torch.argmax(softmaxed_x, dim=0)

for i in range(topk_indices.size(0)):
    if 13 in top3_indices_after_softmax[:, i]:
        print(topk_indices[i])

# Attribution attempt

In [ ]:
from dataclasses import dataclass
from functools import partial
from typing import Any, Literal, NamedTuple, Callable

import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint
import torch.nn.functional as F

# Use the functional API with inplace=False
# feature_acts = self.hook_sae_acts_post(F.relu(hidden_pre, inplace=False))

class SaeReconstructionCache(NamedTuple):
    sae_in: torch.Tensor
    feature_acts: torch.Tensor
    sae_out: torch.Tensor
    sae_error: torch.Tensor


def track_grad(tensor: torch.Tensor) -> None:
    """wrapper around requires_grad and retain_grad"""
    tensor.requires_grad_(True)
    tensor.retain_grad()


@dataclass
class ApplySaesAndRunOutput:
    model_output: torch.Tensor
    model_activations: dict[str, torch.Tensor]
    sae_activations: dict[str, SaeReconstructionCache]

    def zero_grad(self) -> None:
        """Helper to zero grad all tensors in this object."""
        self.model_output.grad = None
        for act in self.model_activations.values():
            act.grad = None
        for cache in self.sae_activations.values():
            cache.sae_in.grad = None
            cache.feature_acts.grad = None
            cache.sae_out.grad = None
            cache.sae_error.grad = None


def apply_saes_and_run(
    model: HookedTransformer,
    saes: dict[str, SAE],
    input: Any,
    include_error_term: bool = True,
    track_model_hooks: list[str] | None = None,
    return_type: Literal["logits", "loss"] = "logits",
    track_grads: bool = False,
) -> ApplySaesAndRunOutput:
    """
    Apply the SAEs to the model at the specific hook points, and run the model.
    By default, this will include a SAE error term which guarantees that the SAE
    will not affect model output. This function is designed to work correctly with
    backprop as well, so it can be used for gradient-based feature attribution.

    Args:
        model: the model to run
        saes: the SAEs to apply
        input: the input to the model
        include_error_term: whether to include the SAE error term to ensure the SAE doesn't affect model output. Default True
        track_model_hooks: a list of hook points to record the activations and gradients. Default None
        return_type: this is passed to the model.run_with_hooks function. Default "logits"
        track_grads: whether to track gradients. Default False
    """

    fwd_hooks = []
    bwd_hooks = []

    sae_activations: dict[str, SaeReconstructionCache] = {}
    model_activations: dict[str, torch.Tensor] = {}

    # this hook just track the SAE input, output, features, and error. If `track_grads=True`, it also ensures
    # that requires_grad is set to True and retain_grad is called for intermediate values.
    def reconstruction_hook(sae_in: torch.Tensor, hook: HookPoint, hook_point: str):  # noqa: ARG001
        sae = saes[hook_point]
#         x = sae_in.to(sae.dtype)
#         x = sae.reshape_fn_in(x)
#         x = sae.hook_sae_input(x)
#         x = sae.run_time_activation_norm_fn_in(x)

#         sae_in = x - (sae.b_dec * sae.cfg.apply_b_dec_to_input)

#         # # "... d_in, d_in d_sae -> ... d_sae",
#         hidden_pre = sae.hook_sae_acts_pre(sae_in @ sae.W_enc + sae.b_enc)
#         feature_acts = sae.hook_sae_acts_post(F.relu(hidden_pre, inplace=False))
        feature_acts = sae.encode(sae_in)
        
        
        sae_out = sae.decode(feature_acts)
        sae_error = (sae_in - sae_out).detach().clone()
        if track_grads:
            track_grad(sae_error)
            track_grad(sae_out)
            track_grad(feature_acts)
            track_grad(sae_in)
        sae_activations[hook_point] = SaeReconstructionCache(
            sae_in=sae_in,
            feature_acts=feature_acts,
            sae_out=sae_out,
            sae_error=sae_error,
        )

        if include_error_term:
            return sae_out + sae_error
        return sae_out

    def sae_bwd_hook(output_grads: torch.Tensor, hook: HookPoint):  # noqa: ARG001
        # this just passes the output grads to the input, so the SAE gets the same grads despite the error term hackery
        return (output_grads,)

    # this hook just records model activations, and ensures that intermediate activations have gradient tracking turned on if needed
    def tracking_hook(hook_input: torch.Tensor, hook: HookPoint, hook_point: str):  # noqa: ARG001
        model_activations[hook_point] = hook_input
        if track_grads:
            track_grad(hook_input)
        return hook_input

    for hook_point in saes.keys():
        fwd_hooks.append(
            (hook_point, partial(reconstruction_hook, hook_point=hook_point))
        )
        bwd_hooks.append((hook_point, sae_bwd_hook))
    for hook_point in track_model_hooks or []:
        fwd_hooks.append((hook_point, partial(tracking_hook, hook_point=hook_point)))

    # now, just run the model while applying the hooks
    with model.hooks(fwd_hooks=fwd_hooks, bwd_hooks=bwd_hooks):
        model_output = model(input, return_type=return_type)

    return ApplySaesAndRunOutput(
        model_output=model_output,
        model_activations=model_activations,
        sae_activations=sae_activations,
    )


from dataclasses import dataclass
from transformer_lens.hook_points import HookPoint
from dataclasses import dataclass
from functools import partial
from typing import Any, Literal, NamedTuple

import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint
torch.autograd.set_detect_anomaly(True)
EPS = 1e-8

torch.set_grad_enabled(True)
@dataclass
class AttributionGrads:
    metric: torch.Tensor
    model_output: torch.Tensor
    model_activations: dict[str, torch.Tensor]
    sae_activations: dict[str, SaeReconstructionCache]


@dataclass
class Attribution:
    model_attributions: dict[str, torch.Tensor]
    model_activations: dict[str, torch.Tensor]
    model_grads: dict[str, torch.Tensor]
    sae_feature_attributions: dict[str, torch.Tensor]
    sae_feature_activations: dict[str, torch.Tensor]
    sae_feature_grads: dict[str, torch.Tensor]
    sae_errors_attribution_proportion: dict[str, float]


def calculate_attribution_grads(
    model: HookedSAETransformer,
    prompt: str,
    metric_fn: Callable[[torch.Tensor], torch.Tensor],
    track_hook_points: list[str] | None = None,
    include_saes: dict[str, SAE] | None = None,
    return_logits: bool = True,
    include_error_term: bool = True,
) -> AttributionGrads:
    """
    Wrapper around apply_saes_and_run that calculates gradients wrt to the metric_fn.
    Tracks grads for both SAE feature and model neurons, and returns them in a structured format.
    """
    output = apply_saes_and_run(
        model,
        saes=include_saes or {},
        input=prompt,
        return_type="logits" if return_logits else "loss",
        track_model_hooks=track_hook_points,
        include_error_term=include_error_term,
        track_grads=True,
    )
    metric = metric_fn(output.model_output)
    output.zero_grad()
    metric.backward()
    return AttributionGrads(
        metric=metric,
        model_output=output.model_output,
        model_activations=output.model_activations,
        sae_activations=output.sae_activations,
    )


def calculate_feature_attribution(
    model: HookedSAETransformer,
    input: Any,
    metric_fn: Callable[[torch.Tensor], torch.Tensor],
    track_hook_points: list[str] | None = None,
    include_saes: dict[str, SAE] | None = None,
    return_logits: bool = True,
    include_error_term: bool = True,
) -> Attribution:
    """
    Calculate feature attribution for SAE features and model neurons following
    the procedure in https://transformer-circuits.pub/2024/march-update/index.html#feature-heads.
    This include the SAE error term by default, so inserting the SAE into the calculation is
    guaranteed to not affect the model output. This can be disabled by setting `include_error_term=False`.

    Args:
        model: The model to calculate feature attribution for.
        input: The input to the model.
        metric_fn: A function that takes the model output and returns a scalar metric.
        track_hook_points: A list of model hook points to track activations for, if desired
        include_saes: A dictionary of SAEs to include in the calculation. The key is the hook point to apply the SAE to.
        return_logits: Whether to return the model logits or loss. This is passed to TLens, so should match whatever the metric_fn expects (probably logits)
        include_error_term: Whether to include the SAE error term in the calculation. This is recommended, as it ensures that the SAE will not affecting the model output.
    """
    # first, calculate gradients wrt to the metric_fn.
    # these will be multiplied with the activation values to get the attributions
    outputs_with_grads = calculate_attribution_grads(
        model,
        input,
        metric_fn,
        track_hook_points,
        include_saes=include_saes,
        return_logits=return_logits,
        include_error_term=include_error_term,
    )
    model_attributions = {}
    model_activations = {}
    model_grads = {}
    sae_feature_attributions = {}
    sae_feature_activations = {}
    sae_feature_grads = {}
    sae_error_proportions = {}
    # this code is long, but all it's doing is multiplying the grads by the activations
    # and recording grads, acts, and attributions in dictionaries to return to the user
    with torch.no_grad():
        for name, act in outputs_with_grads.model_activations.items():
            assert act.grad is not None
            raw_activation = act.detach().clone()
            model_attributions[name] = (act.grad * raw_activation).detach().clone()
            model_activations[name] = raw_activation
            model_grads[name] = act.grad.detach().clone()
        for name, act in outputs_with_grads.sae_activations.items():
            assert act.feature_acts.grad is not None
            assert act.sae_out.grad is not None
            raw_activation = act.feature_acts.detach().clone()
            sae_feature_attributions[name] = (
                (act.feature_acts.grad * raw_activation).detach().clone()
            )
            sae_feature_activations[name] = raw_activation
            sae_feature_grads[name] = act.feature_acts.grad.detach().clone()
            if include_error_term:
                assert act.sae_error.grad is not None
                error_grad_norm = act.sae_error.grad.norm().item()
            else:
                error_grad_norm = 0
            sae_out_norm = act.sae_out.grad.norm().item()
            sae_error_proportions[name] = error_grad_norm / (
                sae_out_norm + error_grad_norm + EPS
            )
        return Attribution(
            model_attributions=model_attributions,
            model_activations=model_activations,
            model_grads=model_grads,
            sae_feature_attributions=sae_feature_attributions,
            sae_feature_activations=sae_feature_activations,
            sae_feature_grads=sae_feature_grads,
            sae_errors_attribution_proportion=sae_error_proportions,
        )
        
        
# prompt = " Tiger Woods plays the sport of"
# pos_token = model.tokenizer.encode(" golf")[0]
prompt = "What is the output of 54 plus 32 ? It is "
pos_token = [model.tokenizer.encode("8")[1]]
neg_token = [model.tokenizer.encode("1")[1]]
# def metric_fn(logits: torch.tensor, pos_token: torch.tensor =pos_token, neg_token: torch.Tensor=neg_token) -> torch.Tensor:
#     return logits[0,-1,pos_token] - logits[0,-1,neg_token]

def metric_fn(logits: torch.Tensor, pos_token: torch.Tensor = pos_token, neg_token: torch.Tensor = neg_token) -> torch.Tensor:
    return (logits[0, -1, pos_token] - logits[0, -1, neg_token]).sum()


feature_attribution_df = calculate_feature_attribution(
    input = prompt,
    model = model,
    metric_fn = metric_fn,
    include_saes={sae.cfg.hook_name: sae},
    include_error_term=True,
    return_logits=True,
)


In [ ]:
def convert_sparse_feature_to_long_df(sparse_tensor: torch.Tensor) -> pd.DataFrame:
    """
    Convert a sparse tensor to a long format pandas DataFrame.
    """
    df = pd.DataFrame(sparse_tensor.detach().cpu().numpy())
    df_long = df.melt(ignore_index=False, var_name='column', value_name='value')
    df_long.columns = ["feature", "attribution"]
    df_long_nonzero = df_long[df_long['attribution'] != 0]
    df_long_nonzero = df_long_nonzero.reset_index().rename(columns={'index': 'position'})
    return df_long_nonzero

df_long_nonzero = convert_sparse_feature_to_long_df(feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0])
df_long_nonzero.sort_values("attribution", ascending=False).head(15)

In [ ]:
for i, v in df_long_nonzero.query("position==12").groupby("feature").attribution.sum().sort_values(ascending=False).head(5).items():
    print(f"Feature {i} had a total attribution of {v:.2f}")
    html = get_dashboard_html(sae_release = "gemma-2-2b", sae_id="8-gemmascope-res-16k", feature_idx=int(i))
    display(IFrame(html, width=1200, height=300))

# Misc

In [ ]:
import re
from collections import defaultdict

sae_keys = list(df.loc['gemma-scope-2b-pt-res']['saes_map'].keys())
# Dictionary to store the closest string for each layer
closest_strings = {}

# Regular expression to extract the layer number and l0 value
pattern = re.compile(r'layer_(\d+)/width_16k/average_l0_(\d+)')

# Organize strings by layer
layer_dict = defaultdict(list)

for s in sae_keys:
    match = pattern.search(s)
    if match:
        layer = int(match.group(1))
        l0_value = int(match.group(2))
        layer_dict[layer].append((s, l0_value))

# Find the string with l0 value closest to 100 for each layer
for layer, items in layer_dict.items():
    closest_string = min(items, key=lambda x: abs(x[1] - 100))
    closest_strings[layer] = closest_string[0]

# Output the closest string for each layer
for layer in sorted(closest_strings):
    print(f"Layer {layer}: {closest_strings[layer]}")


Layer 0: layer_0/width_16k/average_l0_105
Layer 1: layer_1/width_16k/average_l0_102
Layer 2: layer_2/width_16k/average_l0_141
Layer 3: layer_3/width_16k/average_l0_59
Layer 4: layer_4/width_16k/average_l0_124
Layer 5: layer_5/width_16k/average_l0_68
Layer 6: layer_6/width_16k/average_l0_70
Layer 7: layer_7/width_16k/average_l0_69
Layer 8: layer_8/width_16k/average_l0_71
Layer 9: layer_9/width_16k/average_l0_73
Layer 10: layer_10/width_16k/average_l0_77
Layer 11: layer_11/width_16k/average_l0_80
Layer 12: layer_12/width_16k/average_l0_82
Layer 13: layer_13/width_16k/average_l0_84
Layer 14: layer_14/width_16k/average_l0_84
Layer 15: layer_15/width_16k/average_l0_78
Layer 16: layer_16/width_16k/average_l0_78
Layer 17: layer_17/width_16k/average_l0_77
Layer 18: layer_18/width_16k/average_l0_74
Layer 19: layer_19/width_16k/average_l0_73
Layer 20: layer_20/width_16k/average_l0_71
Layer 21: layer_21/width_16k/average_l0_70
Layer 22: layer_22/width_16k/average_l0_72
Layer 23: layer_23/width_16k/average_l0_75
Layer 24: layer_24/width_16k/average_l0_73
Layer 25: layer_25/width_16k/average_l0_116

tensor(4909, device='cuda:0')
tensor(7759, device='cuda:0')
tensor(2003, device='cuda:0')
tensor(4781, device='cuda:0')
tensor(4646, device='cuda:0')
tensor(8109, device='cuda:0')
tensor(2165, device='cuda:0')
tensor(10524, device='cuda:0')
tensor(14888, device='cuda:0')
tensor(15075, device='cuda:0')
tensor(778, device='cuda:0')
tensor(15191, device='cuda:0')
tensor(2707, device='cuda:0')
tensor(10585, device='cuda:0')
tensor(2121, device='cuda:0')
tensor(9261, device='cuda:0')
tensor(4978, device='cuda:0')
tensor(8262, device='cuda:0')
tensor(1083, device='cuda:0')
tensor(571, device='cuda:0')
tensor(9188, device='cuda:0')
tensor(9561, device='cuda:0')
tensor(7876, device='cuda:0')
tensor(14993, device='cuda:0')
tensor(2288, device='cuda:0')
tensor(11746, device='cuda:0')
tensor(5864, device='cuda:0')
tensor(11973, device='cuda:0')
tensor(498, device='cuda:0')
tensor(4788, device='cuda:0')
tensor(5821, device='cuda:0')
tensor(10971, device='cuda:0')
tensor(10825, device='cuda:0')
tensor(5326, device='cuda:0')
